In [1]:
import sys

import pandas as pd
from pandas import DataFrame, Series

import numpy as np

import matplotlib.pyplot as plt
import matplotlib

#import xgboost as xgb

%matplotlib inline

In [3]:
train = pd.read_csv("../../data/train.csv", index_col=0)
test = pd.read_csv("../../data/test.csv", index_col=0)
train.head(2)

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby


In [4]:
test.head(2)

,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
ID,,,,,,,
1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan


In [5]:
data.SexuponOutcome.value_counts()

Neutered Male    9779
Spayed Female    8820
Intact Male      3525
Intact Female    3511
Unknown          1093
Name: SexuponOutcome, dtype: int64

In [7]:
data[data.SexuponOutcome == "Unknown"].AnimalType.value_counts()

Cat    1016
Dog      77
Name: AnimalType, dtype: int64

In [12]:
opt = data
opt.SexuponOutcome = opt.SexuponOutcome.replace("Neutered Male", 1)
opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Male", 1)
opt.SexuponOutcome = opt.SexuponOutcome.replace("Spayed Female", 2)
opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Female", 2)
opt.SexuponOutcome = opt.SexuponOutcome.replace("Unknown", 3)

opt.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,1,1.0,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,2,2.0,1 year,Domestic Shorthair Mix,Cream Tabby
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,1,1.0,2 years,Pit Bull Mix,Blue/White
A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,2,1.0,3 weeks,Domestic Shorthair Mix,Blue Cream
A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,1,1.0,2 years,Lhasa Apso/Miniature Poodle,Tan


In [11]:
opt.AnimalType = opt.AnimalType.replace("Dog", 1)
opt.AnimalType = opt.AnimalType.replace("Cat", 2)